Aim: initial size = 10, find setting AL is superior than IID, use default lr
<br> 
Try: (1) n = 1, mini-batches

In [1]:
import numpy as np
import torch
from torch import nn, distributions
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision import datasets
from dataclasses import dataclass

In [2]:
mnist_train = datasets.MNIST("../data", train=True, download=True, transform=ToTensor())
mnist_test  = datasets.MNIST("../data", train=False,download=True, transform=ToTensor())
traindataloader = DataLoader(mnist_train, shuffle=True, batch_size=60000)
testdataloader  = DataLoader(mnist_test , shuffle=True, batch_size=10000)
X_train, y_train = next(iter(traindataloader))
X_test , y_test  = next(iter(testdataloader))

X_train = X_train.reshape(60000, -1)
X_test = X_test.reshape(10000, -1)

device = "cuda" if torch.cuda.is_available() else "cpu"

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# sepeate train data into (1) initial, (2) pool
initial_idx = np.array([],dtype=int)
for i in range(10):
    idx = np.random.choice(np.where(y_train==i)[0], size=1, replace=False)
    initial_idx = np.concatenate((initial_idx, idx))

X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

X_pool = np.delete(X_train, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)

In [ ]:
# BALD Acquisition function (incomplete sampling in paper)
def bald(model_A, model_S1, model_S2, X, n_instances, T = 100):
  
    random_subset = np.random.choice(range(len(X)), size=len(X), replace=False)
    with torch.no_grad():
        x = X[random_subset].to(device)
        outputs = np.stack([torch.exp(model_A(x)).detach().cpu().numpy() for t in range(T)])

    pc = outputs.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(outputs * np.log(outputs + 1e-10), axis=-1), axis=0) 
    acquisition = H - E_H
    idx = (-acquisition).argsort()[:n_instances]
    query_idx = random_subset[idx]

    score_A = acquisition[query_idx]
    
    #---------------- score for successors:
    with torch.no_grad():
        output_S1 = np.stack([torch.exp(model_S1(X[query_idx].to(device))).detach().cpu().numpy() for t in range(T)])
        output_S2 = np.stack([torch.exp(model_S2(X[query_idx].to(device))).detach().cpu().numpy() for t in range(T)])

    pc = output_S1.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(output_S1 * np.log(output_S1 + 1e-10), axis=-1), axis=0) 
    score_S1 = H - E_H

    pc = output_S2.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(output_S2 * np.log(output_S2 + 1e-10), axis=-1), axis=0) 
    score_S2 = H - E_H

    return (query_idx, score_A, score_S1, score_S2) 

In [4]:
# BALD Acquisition function (in use)
def bald(model_A, model_S1, model_S2, X, n_instances, T = 100):
    with torch.no_grad():
        x = X.to(device)
        outputs = np.stack([torch.exp(model_A(x, eval = True)).detach().cpu().numpy() for t in range(T)])

    pc = outputs.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(outputs * np.log(outputs + 1e-10), axis=-1), axis=0) 
    acquisition = H - E_H
    query_idx = (-acquisition).argsort()[:n_instances]

    score_A = acquisition[query_idx]
    
    #---------------- score for successors:
    with torch.no_grad():
        output_S1 = np.stack([torch.exp(model_S1(X[query_idx].to(device), eval = True)).detach().cpu().numpy() for t in range(T)])
        output_S2 = np.stack([torch.exp(model_S2(X[query_idx].to(device), eval = True)).detach().cpu().numpy() for t in range(T)])

    pc = output_S1.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(output_S1 * np.log(output_S1 + 1e-10), axis=-1), axis=0) 
    score_S1 = H - E_H

    pc = output_S2.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(output_S2 * np.log(output_S2 + 1e-10), axis=-1), axis=0) 
    score_S2 = H - E_H

    return (query_idx, score_A, score_S1, score_S2) 

In [5]:
# IID acquistion:
def iid_acq(model_A, model_S1, model_S2, X, n_instances, T = 100):

    query_idx = np.random.choice(range(len(X)), size=n_instances, replace=False)
    with torch.no_grad():
        x = X[query_idx].to(device)
        outputs = np.stack([torch.exp(model_A(x, eval = True)).detach().cpu().numpy() for t in range(T)])
        output_S1 = np.stack([torch.exp(model_S1(x, eval = True)).detach().cpu().numpy() for t in range(T)])
        output_S2 = np.stack([torch.exp(model_S2(x, eval = True)).detach().cpu().numpy() for t in range(T)])

    pc = outputs.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(outputs * np.log(outputs + 1e-10), axis=-1), axis=0) 
    score_A = H - E_H
    
    #---------------- score for the other 2 models:
    pc = output_S1.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(output_S1 * np.log(output_S1 + 1e-10), axis=-1), axis=0) 
    score_S1 = H - E_H

    pc = output_S2.mean(axis=0)
    H   = (-pc*np.log(pc + 1e-10)).sum(axis=-1)
    E_H = - np.mean(np.sum(output_S2 * np.log(output_S2 + 1e-10), axis=-1), axis=0) 
    score_S2 = H - E_H

    return (query_idx, score_A, score_S1, score_S2) 

In [127]:
def active_learning_procedure(X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              n_queries=100, 
                              n_instances=1,
                              sample_strategy = 'bald',
                              Model_A = 1,
                              Model_S1 = 2,
                              Model_S2 = 3):

    pool_idx = list()
    train_ls = list()
    x_train_sample = X_initial
    y_train_sample = y_initial
    train_loader = DataLoader(list(zip(x_train_sample,  y_train_sample)), 
                      shuffle=True, batch_size = 10)

    #valid_idx = np.random.randint(0, len(y_pool), len(x_train_sample))
    #x_valid = X_pool[valid_idx]
    #y_valid = y_pool[valid_idx]

    #valid_loader = DataLoader(list(zip(x_valid,  y_valid)), 
    #                  shuffle=False, batch_size = len(x_valid))
  
    early_epoch = 100
    max_epoch = 500

    #----- Model A --------------------------
    if (Model_A==1):
        A = Model1(784, 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 100
    elif (Model_A ==2 ):
        A = Model2(784, 392, 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 200
    elif (Model_A == 3):
        A = Model3(784, [392, 196], 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 500
    else:
        print('Invalid option for Model_A')
        return
    A.reset_bs()
    optim0 = torch.optim.Adam(A.parameters(), lr=0.001)
    ls, acc = train(A, optim0, max_epoch, train_loader, X_test, y_test, early_epoch, verbose = False)
    acquisition_hist  = [acc]
    #train_ls.append(ls)

    #----- Model S --------------------------
    if (Model_S1 == 1):
        S1 = Model1(784, 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 100
    elif (Model_S1 ==2):
        S1 = Model2(784, 392, 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 200
    else:
        print('Invalid option for Model_S1')
        return
    S1.reset_bs()
    optim1 = torch.optim.Adam(S1.parameters(), lr=0.001)
    _, acc = train(S1, optim1, max_epoch, train_loader, X_test, y_test, early_epoch, verbose = False)
    successor1_hist = [acc]

    if (Model_S2 == 1):
        S2 = Model1(784, 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 100
    elif (Model_S2 ==3):
        S2 = Model3(784, [392, 196], 10, n_batch = len(train_loader)).to(device)
        #early_epoch = 500
    else:
        print('Invalid option for Model_A')
        return
    S2.reset_bs()  
    optim2 = torch.optim.Adam(S2.parameters(), lr=0.001)
    ls, acc = train(S2, optim2, max_epoch,  train_loader, X_test, y_test, early_epoch, verbose = False)
    successor2_hist = [acc]
    train_ls.append(ls)

    scores = list()

    for index in range(n_queries):
        # for incomplete sampling (paper):
        #query_idx = np.concatenate([bald(A, X_pool) for n in range(n_instances)])

        # for complete set sampling:
        if (sample_strategy  == 'bald'):
            query_idx, score_A, score_S1, score_S2 = bald(A, S1, S2, X_pool, n_instances)
        elif (sample_strategy == 'iid'):
            query_idx, score_A, score_S1, score_S2 = iid_acq(A, S1, S2, X_pool, n_instances)
        else:
            print('Unexpected sampling strategy')
            break
        
        # record scores:
        scores.append(np.vstack((score_A, score_S1, score_S2)))

        x_train_sample = torch.vstack((x_train_sample, X_pool[query_idx]))
        y_train_sample = torch.cat((y_train_sample,y_pool[query_idx]))
        train_loader = DataLoader(list(zip(x_train_sample,  y_train_sample)), 
                      shuffle=True, batch_size = 10)

        #valid_idx = np.random.randint(0, len(y_pool), len(x_train_sample))
        #x_valid = X_pool[valid_idx]
        #y_valid = y_pool[valid_idx]
        #valid_loader = DataLoader(list(zip(x_valid,  y_valid)), 
        #              shuffle=False, batch_size = len(x_valid))

        # model A:
        if (Model_A==1):
            A = Model1(784, 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 100
        elif (Model_A ==2 ):
            A = Model2(784, 392, 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 200
        elif (Model_A == 3):
            A = Model3(784, [392, 196], 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 500
        else:
            print('Invalid option for Model_A')
            return
        A.reset_bs()
        optim0 = torch.optim.Adam(A.parameters(), lr=0.001)
        ls, acc = train(A, optim0, max_epoch, train_loader, X_test, y_test, early_epoch, verbose = False)
        acquisition_hist.append(acc)
        #train_ls.append(ls)

        # model S:
        if (Model_S1 == 1):
            S1 = Model1(784, 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 100
        elif (Model_S1 ==2):
            S1 = Model2(784, 392, 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 200
        else:
            print('Invalid option for Model_S1')
            return
        S1.reset_bs()
        optim1 = torch.optim.Adam(S1.parameters(), lr=0.001)
        _, acc = train(S1, optim1, max_epoch,  train_loader, X_test, y_test, early_epoch, verbose = False)
        successor1_hist.append(acc)

        if (Model_S2 == 1):
            S2 = Model1(784, 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 100
        elif (Model_S2 ==3):
            S2 = Model3(784, [392, 196], 10, n_batch = len(train_loader)).to(device)
            #early_epoch = 500
        else:
            print('Invalid option for Model_A')
            return  
        S2.reset_bs()
        optim2 = torch.optim.Adam(S2.parameters(), lr=0.001)
        ls, acc = train(S2, optim2, max_epoch, train_loader, X_test, y_test, early_epoch, verbose = False)
        successor2_hist.append(acc)
        train_ls.append(ls)

        
        # delete queried data from pool:
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)

        print('Query {n}: {acc:0.4f}(Model A)    |     {acc_s1:0.4f}(Model S1) - {acc_s2:0.4f}(Model S2) '.format(n=index + 1, 
                                                                                                                  acc=acquisition_hist[-1], 
                                                                                                                  acc_s1 = successor1_hist[-1], 
                                                                                                                  acc_s2 = successor2_hist[-1]))     
        pool_idx.append(query_idx)
    return(acquisition_hist, successor1_hist, successor2_hist, scores, pool_idx)

In [77]:
class LinearVariational(nn.Module):
    def __init__(self, in_features, out_features, loss_accumulator, batch_num, n_batch, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.include_bias = bias        
        self.loss_accumulator = loss_accumulator
        self.n_batch = n_batch
        self.batch_num = batch_num
        
        if getattr(loss_accumulator, 'accumulated_kl_div', None) is None:
            loss_accumulator.accumulated_kl_div = 0
        if getattr(batch_num, 'accumulated_bs', None) is None:
            batch_num.accumulated_bs = 0

        # mean:
        self.w_mu = nn.Parameter(
            torch.FloatTensor(in_features, out_features).normal_(mean=0, std=0.001)
        )
         
        # variance log(1 + exp(p))◦ eps:
        self.w_p = nn.Parameter(
            torch.FloatTensor(in_features, out_features).normal_(mean=-2.5, std=0.001)
        )
        if self.include_bias:
            self.b_mu = nn.Parameter(
                torch.zeros(out_features)
            )
            self.b_p = nn.Parameter(
                torch.zeros(out_features)
            )
        
    def reparameterize(self, mu, p):
        sigma = torch.log(1 + torch.exp(p)) 
        eps = torch.randn_like(sigma)
        return mu + (eps * sigma)
    
    def kl_divergence(self, z, mu_theta, p_theta, batch_num, prior_sd=1):
        log_prior = distributions.Normal(0, prior_sd).log_prob(z) 
        log_p_q = distributions.Normal(mu_theta, torch.log(1 + torch.exp(p_theta))).log_prob(z) 
        weights =  np.power(2,self.n_batch - batch_num) /  (np.power(2,self.n_batch)-1) 

        #print('batch_num {}: weights {}'.format(batch_num, weights))
        return (log_p_q - log_prior).mean() * weights

    def forward(self, x):
        w = self.reparameterize(self.w_mu, self.w_p)
        
        if self.include_bias:
            b = self.reparameterize(self.b_mu, self.b_p)
        else:
            b = 0
            
        z = x @ w + b
        
        if (self.batch_num.clip == False):
            self.batch_num.accumulated_bs += 1
        self.loss_accumulator.accumulated_kl_div += self.kl_divergence(w, 
                                                             self.w_mu,
                                                             self.w_p,
                                                             self.batch_num.accumulated_bs
                                                             )
        if self.include_bias:
            self.loss_accumulator.accumulated_kl_div += self.kl_divergence(b, 
                                                                 self.b_mu, 
                                                                 self.b_p,
                                                                 self.batch_num.accumulated_bs
                                                                 )
        return z


@dataclass
class KL:
    accumulated_kl_div = 0
class BS:
    accumulated_bs = 0
    clip = False

def det_loss(y, y_pred, model):
    reconstruction_error = F.nll_loss(y_pred, y,reduction="mean")
    kl = model.accumulated_kl_div
    #model.reset_kl_div() # reset fo each batch
    #model.reset_bs()
    #print('rec: {} | kl: {}'.format(reconstruction_error, kl))
    #print('loss {}'.format(reconstruction_error + kl))
    return reconstruction_error + kl



In [40]:
class Model1(nn.Module):
    def __init__(self, in_size, out_size, n_batch):
        super().__init__()
        self.kl_loss = KL
        self.bs_num = BS

        self.var = LinearVariational(in_size, out_size, self.kl_loss, self.bs_num, n_batch)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim = 1)
        
        self.layers = nn.Sequential(
            LinearVariational(in_size, out_size, self.kl_loss, self.bs_num, n_batch),
            nn.LogSoftmax(dim = 1)
        )
    
    @property
    def accumulated_kl_div(self):
        return self.kl_loss.accumulated_kl_div
    
    def reset_kl_div(self):
        self.kl_loss.accumulated_kl_div = 0

    def accumulated_bs(self):
        return self.bs_num.accumulated_bs
    
    def reset_bs(self):
        self.bs_num.accumulated_bs = 0
    
    def clip_bs(self, clip = True):
        self.bs_num.clip = clip
    
    def set_bs(self, val):
        self.bs_num.accumulated_bs = val
            
    def forward(self, x, eval = False):
        if eval:
            Model1.set_bs(self, val = 1)
            Model1.clip_bs(self, clip = True)
        else:
            Model1.clip_bs(self, clip = False)
        out = self.var(x)
        out = self.log_softmax(out)
        return out

class Model2(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_batch):
        super().__init__()
        self.kl_loss = KL
        self.bs_num = BS
        
        self.var1 = LinearVariational(in_size, hidden_size, self.kl_loss, self.bs_num, n_batch)
        self.lin1 = nn.Linear(in_size, hidden_size)
        self.var2 = LinearVariational(hidden_size, out_size, self.kl_loss, self.bs_num, n_batch)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim = 1)


    @property
    def accumulated_kl_div(self):
        return self.kl_loss.accumulated_kl_div
    
    def reset_kl_div(self):
        self.kl_loss.accumulated_kl_div = 0

    def accumulated_bs(self):
        return self.bs_num.accumulated_bs
    
    def reset_bs(self):
        self.bs_num.accumulated_bs = 0

    def clip_bs(self, clip = True):
        self.bs_num.clip = clip
    
    def set_bs(self, val):
        self.bs_num.accumulated_bs = val
            
    def forward(self, x, eval = False):
        if eval:
            Model2.set_bs(self, val = 1)
            Model2.clip_bs(self, clip = True)
        else:
            Model2.clip_bs(self, clip = False)
        #out = self.var1(x)   
        out = self.lin1(x)
        out = self.relu(out)

        #Model2.clip_bs(self)

        out = self.var2(out)
        out = self.log_softmax(out)


        return out

class Model3(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_batch):
        super().__init__()
        self.kl_loss = KL
        self.bs_num = BS

        self.var1 = LinearVariational(in_size, hidden_size[0], self.kl_loss, self.bs_num, n_batch)
        self.var2 = LinearVariational(hidden_size[0], hidden_size[1], self.kl_loss, self.bs_num, n_batch)
        self.var3 = LinearVariational(hidden_size[1], out_size, self.kl_loss, self.bs_num, n_batch)

        self.lin1 = nn.Linear(in_size, hidden_size[0])
        self.lin2 = nn.Linear(hidden_size[0], hidden_size[1])

        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim = 1)

    
    @property
    def accumulated_kl_div(self):
        return self.kl_loss.accumulated_kl_div
    
    def reset_kl_div(self):
        self.kl_loss.accumulated_kl_div = 0

    def accumulated_bs(self):
        return self.bs_num.accumulated_bs
    
    def reset_bs(self):
        self.bs_num.accumulated_bs = 0

    def clip_bs(self, clip = True):
        self.bs_num.clip = clip

    def set_bs(self, val):
        self.bs_num.accumulated_bs = val
            
    def forward(self, x, eval = False):
        if eval:
            Model3.set_bs(self, val = 1)
            Model3.clip_bs(self, clip = True)
        else:
            Model3.clip_bs(self, clip = False)
        #out = self.var1(x)
        out = self.lin1(x)
        out = self.relu(out)

        #Model3.clip_bs(self)
        #out = self.var2(out)
        out = self.lin2(out)
        out = self.relu(out)

        out = self.var3(out)
        out = self.log_softmax(out)
        return out

In [112]:
# Train
def train(model, optimizer, num_epoch,
          train_loader, 
          X_test, y_test, early_epoch,
          verbose = False):
    train_ls = list()
    #valid_ls = list()
    test_acc = 0
    train_kl = list()

    for epoch in range(num_epoch):
        ls = 0
        kl = 0
        num_correct = 0
        data_num = 0

        model.train()
        for batch_num, (x, y) in enumerate(train_loader):
            #print('in train : batch_num{}'.format(batch_num))

            x = x.to(device)
            y = y.to(device)

            # forward:
            y_pred = model(x, eval = False)
            loss = det_loss(y, y_pred, model)
            kl += model.accumulated_kl_div
            model.reset_kl_div()
            ls += loss.item()

            # backward:
            optimizer.zero_grad() # remove grad from forward prop
            loss.backward()

            # update gradient:
            optimizer.step()

            # accuracy:
            #_, pred = torch.max(y_pred,1)
            #num_correct += (pred == y).sum().item()

            data_num += len(y)
        #model.reset_bs()

        #ls = ls/data_num
        #print('valid:')
        #val_ls = eval_ls(model, valid_loader)
        #weights = np.power(2, len(train_loader)-1) / (np.power(2, len(train_loader)) - 1 )
        #weights = 1

        #val_ls = val_ls/weights
        model.reset_bs()
        if verbose:
            print(epoch+1)
            print(f'\tLoss: {ls:.4f}(train)\t')
        train_ls.append(ls)
        #valid_ls.append(val_ls)
        train_kl.append(kl)

        # access early stopping:
        #trigger = (np.array(train_ls[epoch-50:]) < np.array(valid_ls[epoch-50:])).sum()/500
        #if (trigger > 0.5):
        #    if verbose:
        #        print('Epoch {} : Early Stopping - past 50 epochs half val_ls > train_ls'.format(epoch))
        #    test_acc = eval_acc(model, X_test, y_test)
        #    break

        if ((epoch > early_epoch) & (epoch % 10 == 0)):
            mean_1 = np.mean(train_ls[epoch-60:epoch-10])
            mean_2 = np.mean(train_ls[epoch-50:])
            if (mean_1 - mean_2 < 1e-3):
                print(epoch)
                test_acc = eval_acc(model, X_test, y_test)
                break

        if (epoch == num_epoch-1):
            print(epoch)
            test_acc = eval_acc(model, X_test, y_test)
        

    return(train_ls, test_acc)


def eval_ls(model, loader):
    ls = 0
    data_num = 0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            y_pred = model(x, eval = True)
            loss = det_loss(y, y_pred, model)
            ls += loss.item()

            data_num += len(y)
        model.reset_bs()
    model.train()
    return ls/data_num

def eval_acc(model, test_x, test_y):

    y_sample = np.stack([torch.exp(model(test_x.to(device), eval = True)).detach().cpu().numpy() for t in range(100)])

    y_pred = y_sample.mean(axis = 0).argmax(axis = 1)

    #y_sample = model(test_x.to(device)).detach().cpu().numpy()
    #y_pred = y_sample.argmax(axis = 1)
    acc = np.equal(y_pred, test_y).sum().item()/len(test_y)
    return acc


In [143]:
# A1 BALD
acquisition_hist, successor1_hist, successor2_hist, scores, pool_idx = active_learning_procedure(
                                                                                X_test,
                                                                                y_test,
                                                                                X_pool,
                                                                                y_pool,
                                                                                X_initial,
                                                                                y_initial,
                                                                                n_queries = 50,
                                                                                n_instances = 1,
                                                                                Model_A = 1, Model_S1 = 2, Model_S2 = 3)
with open('res_bald_A1.npy', 'wb') as f:
    np.save(f, acquisition_hist)
    np.save(f, successor1_hist)
    np.save(f, successor2_hist)
    np.save(f, scores)

from google.colab import files
files.download('res_bald_A1.npy')


340
160
150
450
499
240
Query 1: 0.4710(Model A)    |     0.5095(Model S1) - 0.4993(Model S2) 
460
240
260
Query 2: 0.4704(Model A)    |     0.4852(Model S1) - 0.4505(Model S2) 
499
499
390
Query 3: 0.4864(Model A)    |     0.4998(Model S1) - 0.4381(Model S2) 
499
280
499
Query 4: 0.4514(Model A)    |     0.4704(Model S1) - 0.4573(Model S2) 
499
260
220
Query 5: 0.4675(Model A)    |     0.5027(Model S1) - 0.4568(Model S2) 
499
270
300
Query 6: 0.4924(Model A)    |     0.4950(Model S1) - 0.5104(Model S2) 
499
170
200
Query 7: 0.5233(Model A)    |     0.5250(Model S1) - 0.5129(Model S2) 
499
270
160
Query 8: 0.5396(Model A)    |     0.5463(Model S1) - 0.5422(Model S2) 
499
210
470
Query 9: 0.5408(Model A)    |     0.5322(Model S1) - 0.5251(Model S2) 
499
180
170
Query 10: 0.5386(Model A)    |     0.5439(Model S1) - 0.5087(Model S2) 
499
270
360
Query 11: 0.5471(Model A)    |     0.5576(Model S1) - 0.5204(Model S2) 
499
499
499
Query 12: 0.5626(Model A)    |     0.5446(Model S1) - 0.5230(

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [140]:
#iid
acquisition_hist, successor1_hist, successor2_hist, scores, pool_idx = active_learning_procedure(
                                                                                X_test,
                                                                                y_test,
                                                                                X_pool,
                                                                                y_pool,
                                                                                X_initial,
                                                                                y_initial,
                                                                                n_queries = 50,
                                                                                n_instances = 1,
                                                                                sample_strategy = 'iid',
                                                                                Model_A = 1, Model_S1 = 2, Model_S2 = 3)
with open('res_iid_A1.npy', 'wb') as f:
    np.save(f, acquisition_hist)
    np.save(f, successor1_hist)
    np.save(f, successor2_hist)
    np.save(f, scores)

from google.colab import files
files.download('res_iid_A1.npy')

200
140
200
499
200
310
Query 1: 0.4852(Model A)    |     0.5020(Model S1) - 0.4912(Model S2) 
370
220
180
Query 2: 0.4697(Model A)    |     0.4789(Model S1) - 0.4890(Model S2) 
370
170
430
Query 3: 0.4763(Model A)    |     0.4925(Model S1) - 0.5109(Model S2) 
330
240
250
Query 4: 0.4412(Model A)    |     0.4515(Model S1) - 0.4570(Model S2) 
470
330
230
Query 5: 0.4618(Model A)    |     0.4657(Model S1) - 0.4477(Model S2) 
499
220
170
Query 6: 0.4905(Model A)    |     0.5034(Model S1) - 0.5278(Model S2) 
499
320
190
Query 7: 0.5015(Model A)    |     0.5070(Model S1) - 0.5266(Model S2) 
499
230
230
Query 8: 0.5166(Model A)    |     0.5348(Model S1) - 0.5483(Model S2) 
499
499
350
Query 9: 0.5255(Model A)    |     0.5386(Model S1) - 0.5140(Model S2) 
499
480
470
Query 10: 0.5325(Model A)    |     0.5323(Model S1) - 0.5307(Model S2) 
499
499
190
Query 11: 0.5113(Model A)    |     0.5238(Model S1) - 0.5354(Model S2) 
499
499
499
Query 12: 0.5159(Model A)    |     0.5217(Model S1) - 0.5278(

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [141]:
# A2 BALD
acquisition_hist, successor1_hist, successor2_hist, scores, pool_idx = active_learning_procedure(
                                                                                X_test,
                                                                                y_test,
                                                                                X_pool,
                                                                                y_pool,
                                                                                X_initial,
                                                                                y_initial,
                                                                                n_queries = 50,
                                                                                n_instances = 1,
                                                                                Model_A = 2, Model_S1 = 1, Model_S2 = 3)
with open('res_bald_A2.npy', 'wb') as f:
    np.save(f, acquisition_hist)
    np.save(f, successor1_hist)
    np.save(f, successor2_hist)
    np.save(f, scores)

from google.colab import files
files.download('res_bald_A2.npy')

200
280
160
350
380
200
Query 1: 0.5045(Model A)    |     0.4993(Model S1) - 0.4180(Model S2) 
499
499
499
Query 2: 0.4743(Model A)    |     0.4733(Model S1) - 0.4707(Model S2) 
390
470
499
Query 3: 0.4792(Model A)    |     0.4483(Model S1) - 0.4383(Model S2) 
180
499
190
Query 4: 0.4632(Model A)    |     0.4494(Model S1) - 0.4320(Model S2) 
180
499
230
Query 5: 0.4313(Model A)    |     0.4163(Model S1) - 0.4444(Model S2) 
210
499
499
Query 6: 0.4071(Model A)    |     0.3795(Model S1) - 0.4165(Model S2) 
310
480
190
Query 7: 0.4354(Model A)    |     0.4273(Model S1) - 0.3993(Model S2) 
190
270
499
Query 8: 0.4427(Model A)    |     0.4201(Model S1) - 0.4479(Model S2) 
220
499
190
Query 9: 0.4934(Model A)    |     0.4796(Model S1) - 0.4349(Model S2) 
370
499
190
Query 10: 0.4680(Model A)    |     0.4661(Model S1) - 0.4623(Model S2) 
400
499
440
Query 11: 0.4877(Model A)    |     0.4904(Model S1) - 0.4633(Model S2) 
499
499
190
Query 12: 0.4998(Model A)    |     0.4908(Model S1) - 0.4753(

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [142]:
# A3 BALD
acquisition_hist, successor1_hist, successor2_hist, scores, pool_idx = active_learning_procedure(
                                                                                X_test,
                                                                                y_test,
                                                                                X_pool,
                                                                                y_pool,
                                                                                X_initial,
                                                                                y_initial,
                                                                                n_queries = 50,
                                                                                n_instances = 1,
                                                                                Model_A = 3, Model_S1 = 2, Model_S2 = 1)
with open('res_bald_A3.npy', 'wb') as f:
    np.save(f, acquisition_hist)
    np.save(f, successor1_hist)
    np.save(f, successor2_hist)
    np.save(f, scores)

from google.colab import files
files.download('res_bald_A3.npy')


150
140
220
300
250
499
Query 1: 0.5003(Model A)    |     0.5122(Model S1) - 0.5020(Model S2) 
260
340
499
Query 2: 0.4680(Model A)    |     0.4521(Model S1) - 0.4420(Model S2) 
230
499
499
Query 3: 0.4078(Model A)    |     0.4647(Model S1) - 0.4573(Model S2) 
240
240
499
Query 4: 0.4504(Model A)    |     0.4573(Model S1) - 0.4470(Model S2) 
300
250
370
Query 5: 0.4360(Model A)    |     0.4312(Model S1) - 0.4138(Model S2) 
499
290
499
Query 6: 0.4365(Model A)    |     0.4413(Model S1) - 0.4252(Model S2) 
499
499
400
Query 7: 0.4381(Model A)    |     0.4855(Model S1) - 0.4588(Model S2) 
210
220
430
Query 8: 0.4238(Model A)    |     0.4692(Model S1) - 0.4618(Model S2) 
270
180
499
Query 9: 0.4750(Model A)    |     0.4806(Model S1) - 0.4746(Model S2) 
260
220
450
Query 10: 0.5029(Model A)    |     0.4930(Model S1) - 0.4770(Model S2) 
260
210
499
Query 11: 0.4554(Model A)    |     0.4491(Model S1) - 0.4688(Model S2) 
499
499
499
Query 12: 0.4731(Model A)    |     0.4873(Model S1) - 0.4754(

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>